# 1. Preprocessing

In [2]:
source("https://raw.githubusercontent.com/eogasawara/mylibrary/master/myGraphics.R")
#devtools::install_github("bergant/datamodelr")
library("datamodelr")
loadlibrary("DiagrammeR")
loadlibrary("dplyr")
loadlibrary("ggraph")
loadlibrary("igraph")
loadlibrary("ggplot2")
plot_size(4, 3)

ERROR: Error in library("datamodelr"): there is no package called 'datamodelr'


Preprocessing of data is almost always necessary as a prerequisite prior to the processing of some method or methods used in data science, such as machine learning, network analysis, among others.

The need is generated by the fact that in most cases, for the purpose of the analysis, pieces of information contained in several databases are needed. These databases may contain incomplete information or are in undesirable format for the processing itself.

Below is a simple example of preprocessing data using flight information, airports, airlines and weather conditions over a period of time.

Let's say that the purpose of preprocessing is to obtain a concise dataset of flight data containing origin, date, time of delay and indication of weather conditions on the day.

First, we load information from different databases.

In [ ]:
#reads files from airports, airlines, flights and weather conditions.
airlines <- read.csv(file="airline.csv", header=TRUE, sep=",")
airports <- read.csv(file="airport.csv", header=TRUE, sep=",")
flights <- read.csv(file="flights.csv", header=TRUE, sep=",")
weather <- read.csv(file="weather.csv", header=TRUE, sep=",")


head(airlines)
head(airports)
head(flights)
head(weather)

In [ ]:
#mount data model
dm_f <- dm_from_data_frames(flights, airlines, weather, airports)


dm_f <- dm_add_references(
  dm_f,
  
  flights$AIRLINE == airlines$IATA,
  flights$ORIGIN_AIRPORT == airports$IATA_CODE,
  flights$DESTINATION_AIRPORT == airports$IATA_CODE,
  weather$IATA == airports$IATA_CODE
)
graph <- dm_create_graph(dm_f, rankdir = "BT", col_attr = c("column", "type"))
dm_render_graph(graph)


Reduction - In our reduction example, we selected four columns out of the twenty three in the flight database. We selected only the columns that would be interesting for the purpose of the example study, which follow: year, day, month, airline, airport of origin and departure delay.

In [ ]:
#reduction - only use fields for the desired job
red <- data.frame(YEAR = flights$YEAR, MONTH = flights$MONTH, 
                  DAY = flights$DAY, AIRLINE = flights$AIRLINE, 
                  ORIGIN_AIRPORT = flights$ORIGIN_AIRPORT, 
                  DEPARTURE_DELAY = flights$DEPARTURE_DELAY)

head(red)

Cleaning - In our example of cleaning, we disregard all flight records where one of the basic and mandatory information we identified was null. If the information is null, it will not be useful in our example study, so it was discarded. Other types of cleanup could be applied, such as identifying information that does not conform to the expected format or type. For example, a field in which we expect to find a date, however comes full in the base as a list of characters.

In [ ]:
#cleaning - remove any record with at least one of the fields equal to null
clr <- na.omit(red)
head(clr)

Transformation - In our example of transformation, we use some of the fields from the base of climatic conditions and we make a synthetic evaluation of the situation in the day in question. In this case, we analyzed rainfall information on the day, amount of rainfall and fog level to generate a unique climatic condition assessment between the values "ok" and "alert"

In [ ]:
#transformation - to create new dataframe with flight and weather data
trweather <- weather
trweather$condition <- NA
for(i in 1:nrow(trweather)){
  trweather$condition[i] <- if ((trweather$RAINTODAY[i] == "Yes") && (trweather$RAINFALL[i] > 0.2) && (trweather$CLOUD9AM[i] > 7)) {"Alert"} else {"Ok"}
}
head(trweather)

Integration - In our integration example, we integrate the already reduced and clean bases of flights and weather conditions on a single basis that will be the database for processing. We linked the bases to the airport information of origin and date, generating a unique flight view and weather condition on the day.

In [ ]:
#integration - Integrates the bases of flights and climate through the airport code, and date on which the flight occurred. 
mer <- merge(clr, trweather, by.x=c("ORIGIN_AIRPORT","YEAR","MONTH","DAY"), by.y=c("IATA","YEAR","MONTH","DAY"))
basedata <- data.frame(ORIGIN_AIRPORT = mer$ORIGIN_AIRPORT, MONTH =mer$MONTH, 
                       DAY = mer$DAY, YEAR = mer$YEAR, AIRLINE = mer$AIRLINE, 
                       DEPARTURE_DELAY =mer$DEPARTURE_DELAY, WEATHER_CONDITION = mer$condition)

dresult <- dm_from_data_frames(basedata)
graph <- dm_create_graph(dresult, rankdir = "BT", col_attr = c("column", "type"))
dm_render_graph(graph)

basedata

# 2. Clustering

In [ ]:
source("https://raw.githubusercontent.com/eogasawara/mylibrary/master/myGraphics.R")
loadlibrary("caret")
loadlibrary("cluster")
plot_size(4, 3)

Clustering is the set of techniques that allows us to analyze and classify data by similarity / similarity. Clustering is a common technique for market segmentation because it finds similar groups according to the dataset. We can, for example, use grouping for market segmentation dive clients into smaller and more similar groups, and then design the marketing strategy specifically for each group.

For example, an airline is trying to learn more about its customers so that it can segment different customer segments with different types of mileage offerings.

In our example, we set up a synthetic database with three information, namely:

miles = number of miles miles12 = miles in the last 12 months flights = number of flights in the last 12 months daysprogram = days since enrolled the program

In [ ]:
#read source data
mileage <- read.csv("cluster.csv")
summary(mileage)

In [ ]:
head(mileage, 12)

In [ ]:
#normalizing data
preproc <-  preProcess(mileage)
mileageNorm <- predict(preproc, mileage)
summary(mileageNorm)

In [ ]:
#distances between data points (using euclidean distance) 
distances <- dist(mileageNorm, method = "euclidean")

# Hierarchical clustering
mileageClust <- hclust(distances, method = "ward.D") 

plot(mileageClust)

In [ ]:
# K-Means Clustering with 5 clusters
fit <- kmeans(mileageNorm, 5)

# Cluster Plot against 1st 2 principal components

# vary parameters for most readable graph
clusplot(mileageNorm, fit$cluster, color=TRUE, shade=TRUE, 
         labels=2, lines=0)

In our example: Cluster 1 with the largest average values in DaysSinceEnroll. Cluster 2 with the largest average values in miles. Cluster 3 with customers have lower than average values in all variables. Cluster 4 with customers with few miles, but who have been with the airline the longest. Cluster 5 with the greater average values in miles.

# 3. Machine Learning - Neural Network

In [ ]:
source("https://raw.githubusercontent.com/eogasawara/mylibrary/master/myGraphics.R")
library("ggplot2")
library("reshape")
library("base")
library("stats")
library("dplyr")
library("neuralnet")

plot_size(4, 3)

Neural Networks is inspired by the human brain to performs a particular task or functions. Neural Network (or Artificial Neural Network) has the ability to learn by examples. The process of learning occurs through a set of connected input/output units in which each connection has a weight associated with it. In the learning phase, the network learns by adjusting the weights to predict the correct class label of the given inputs. The hidden layer where processing takes place through the application of mathematical functions and weight adjustments needs to be trained so that the results are adequate to the input data.

To iniciate we create a dataset with random information exemplifying flight data. For each of these six flights, the table contains the TTD Time to Departure (original time in minutes marked for departure) and the TSA Time since Arrival (time in minutes since the aircraft arrived on the ground). They are information that, when looking at the input data, we can infer the result, which is not used to train the processing layer.

In [ ]:
# creating trainning data set
ttd=c(20,10,15,20,80,30)
tsa=c(5,30,15,50,50,80)
delayed=c(1,1,1,0,0,0)

# single dataset combining multiple columns
df=data.frame(ttd,tsa,delayed)
df

Next step is to create our Neural Network using the trainning data above (df).

In [ ]:
# neural network
# hidden=3: represents single layer with 3 neurons respectively.
nn=neuralnet(delayed~ttd+tsa,data=df, hidden=3,act.fct = "logistic",
             linear.output = FALSE)
plot(nn)

Now we proccess with the teste data in the trained NN.

In [ ]:
# test data
ttd=c(30,10,20)
tsa=c(85,5,10)
test=data.frame(ttd,tsa)


## Prediction delay for the flights considering the NN trained
Predict=compute(nn,test)
Predict$net.result

# Converting into final results indicating delay (1)yes / (0)no
prob <- Predict$net.result
pred <- ifelse(prob>0.5, 1, 0)
pred

In [ ]:
# creating trainning data set
id=c(1,2,3,4,5,6)
time_to_departure=c(20,10,15,20,80,30)
time_since_arriving=c(5,30,15,50,50,80)
status_on_departure=c("Delayed","Delayed","Delayed","On time","On time","On time")

# single dataset combining multiple columns
df=data.frame(id,time_to_departure,time_since_arriving,status_on_departure)
head(df)

In [ ]:
# creating trainning data set
id=c(7,8,9)
time_to_departure=c(30,10,20)
time_since_arriving=c(85,5,10)
status_on_departure=c("On time","Delayed","Delayed")

# single dataset combining multiple columns
df=data.frame(id,time_to_departure,time_since_arriving,status_on_departure)
head(df)

# 4. Machine Learning - Random Forest

In [ ]:
source("https://raw.githubusercontent.com/eogasawara/mylibrary/master/myGraphics.R")
loadlibrary("randomForest")
loadlibrary("dplyr")
loadlibrary("ggraph")
loadlibrary("igraph")
plot_size(4, 3)

Random Forest combines the output of several decision trees and finally presents its own output. Random Forest selects all data points and variables in each of the trees. It randomly collects data points and variables in each tree it creates and then combines the output at the end. It removes the tendency that a decision tree model can introduce into the system. In addition, it significantly improves predictive power. This follows a gain in the prediction ability in relation to the decision trees by bringing the possibility of combined and random analysis among the analyzes of each tree.

For our example, we created synthetic data from some observed scenarios involving the climatic conditions at the origin of a flight route, the weather conditions at the destination and the indication whether the conditions represented alert or was acceptable to perform the flight without delays.

In [ ]:
# creating data set

#weather on origin
OrigWeather=c('goog', 'vgood', 'bad', 'bad', 'vbad', 'goog', 'bad', 
        'acceptable', 'bad', 'vbad', 'goog', 'vgood', 'acceptable', 'bad', 
        'vbad', 'goog', 'vgood', 'acceptable', 'bad', 'goog', 'goog', 'vgood',
        'acceptable', 'acceptable', 'vbad', 'goog', 'vgood', 'acceptable', 'goog', 
        'goog')

#local weather 
LocalWeather=c('goog', 'vgood', 'acceptable', 'bad', 'vbad', 'goog', 'vgood', 
         'acceptable', 'bad', 'vbad', 'goog', 'vgood', 'acceptable', 'bad', 
         'vbad', 'goog', 'vgood', 'acceptable', 'bad', 'vbad', 'goog', 'vgood',
         'acceptable', 'bad', 'vbad', 'goog', 'vgood', 'acceptable', 'bad', 
         'vbad')

#condition indication
condition = c('acceptable', 'acceptable', 'acceptable', 'adverse', 'adverse', 'acceptable', 'acceptable', 
              'acceptable', 'adverse', 'adverse', 'acceptable', 'acceptable', 'acceptable', 'adverse', 
              'adverse', 'acceptable', 'acceptable', 'acceptable', 'adverse', 'adverse', 'acceptable', 'acceptable',
              'acceptable', 'adverse', 'adverse', 'acceptable', 'acceptable', 'acceptable', 'adverse', 
              'adverse')


# single dataset combining multiple columns
df=data.frame(OrigWeather,LocalWeather, condition)
head(df)

Then we separate the dataset between training data and data that will be tested in the trained model.

In [ ]:
# Split into Train and Validation sets
# Training Set : Validation Set = 70 : 30 (random)
set.seed(30)
train <- sample(nrow(df), 0.7*nrow(df), replace = FALSE)
TrainSet <- df[train,]
ValidSet <- df[-train,]

We create and traine the random forest.

In [ ]:
# Create a Random Forest model with default parameters
model <- randomForest(TrainSet$condition ~ ., data = TrainSet, importance = TRUE)


# Predicting on train set
predTrain <- predict(model, TrainSet, type = "class")
# Checking classification accuracy
table(predTrain, TrainSet$condition)  

In [ ]:
# Predicting on Validation set
predValid <- predict(model, ValidSet, type = "class")
# Checking classification accuracy
table(predValid,ValidSet$condition)

Plot the tree with max number of levels.

In [ ]:
tree_func <- function(final_model, 
                      tree_num) {
  
  # get tree by index
  tree <- randomForest::getTree(final_model, 
                                k = tree_num, 
                                labelVar = TRUE) %>%
    tibble::rownames_to_column() %>%
    # make leaf split points to NA, so the 0s won't get plotted
    mutate(`split point` = ifelse(is.na(prediction), `split point`, NA))
  
  # prepare data frame for graph
  graph_frame <- data.frame(from = rep(tree$rowname, 2),
                            to = c(tree$`left daughter`, tree$`right daughter`))
  
  # convert to graph and delete the last node that we don't want to plot
  graph <- graph_from_data_frame(graph_frame) %>%
    delete_vertices("0")
  
  # set node labels
  V(graph)$node_label <- gsub("_", " ", as.character(tree$`split var`))
  V(graph)$leaf_label <- as.character(tree$prediction)
  V(graph)$split <- as.character(round(tree$`split point`, digits = 2))
  
  # plot
  plot <- ggraph(graph, 'dendrogram') + 
    theme_bw() +
    geom_edge_link() +
    geom_node_point() +
    geom_node_text(aes(label = node_label), na.rm = TRUE, repel = TRUE) +
    geom_node_label(aes(label = split), vjust = 2.5, na.rm = TRUE, fill = "white") +
    geom_node_label(aes(label = leaf_label, fill = leaf_label), na.rm = TRUE, 
                    repel = TRUE, colour = "white", fontface = "bold", show.legend = FALSE) +
    theme(panel.grid.minor = element_blank(),
          panel.grid.major = element_blank(),
          panel.background = element_blank(),
          plot.background = element_rect(fill = "white"),
          panel.border = element_blank(),
          axis.line = element_blank(),
          axis.text.x = element_blank(),
          axis.text.y = element_blank(),
          axis.ticks = element_blank(),
          axis.title.x = element_blank(),
          axis.title.y = element_blank(),
          plot.title = element_text(size = 18))
  
  print(plot)
}


# plot random tree with max number of nodes
treenum = max(model$forest$ndbigtree)
tree_func(model, 500)

# 5. Network Analysis

In [ ]:
source("https://raw.githubusercontent.com/eogasawara/mylibrary/master/myGraphics.R")
library("geosphere")
library("maps")
library("mapproj")
library("ggplot2")
#library("ggnet")
library("igraph")
library("network")
library("base")
library("sna")
library("dplyr")
library("sp")
library("ggraph")
library("GGally")


plot_size(4, 3)

Network representation encompasses the study of flight systems according to a graph theory. Using graph theory terminology, flight networks, for example, can be modeled as a directed graph G(V,E), where V is the set of |V| nodes and E is the set of |E| edges. A node i \in V represents a airport with a connection point. The arcs (i, j) \in E, i \in V and j \in V represent a route between two airports.

One of the basic measures of a vertex is its degree, defined by the number of connections with other vertices in the network, represented by the routes. In our example, the greater the degree of a vertex, the greater the number of routes that pass through that airport, which may have direct implication in its importance for the local loop or the whole system. One of the ways to identify central airports may be by their degree and, from this, take measures to mitigate the effects of a delay or problems at this point, which could impact the whole system.

A well-known measure is the betweenness centrality of a node v. It is a measure aimed at summarizing the extent to which a vertex is located ‘between’ other pairs of vertices. It is a measure that quantifies the number of times an edge, in our case a route, acts as bridge along the shortest path between two other vertices.

In our example, we selected data from the main airports in Brazil, including geo-positioning. From this information, we generate synthetic data simulating routes between nineteen of its main airports. A network was generated from this information, showing how it is placed on the map and its links. We calculate the degree of each airport, given the number of routes it intercepts, and the degree of centrality betweenness, showing which route is most used so that we can arrive at all points of the network.

In [ ]:
#read list of airports in Brazil and ints GIS
airports <- read.csv("brazil_airports.csv", header = TRUE)
airports <- airports[,2:10]
rownames(airports) <- airports$icao
head(airports)

In [ ]:

#read example file with flights with origin and destination airports
routes <- read.csv("routes.csv", header = TRUE)
head(routes)


In [ ]:
# convert to network
routes <- network(routes, directed = TRUE)
plot(routes)


# add geographic coordinates
routes %v% "lat" <- airports[ network.vertex.names(routes), "lat" ]
routes %v% "lon" <- airports[ network.vertex.names(routes), "long" ]


# compute degree centrality
routes %v% "degree" <- degree(routes, gmode = "digraph")

# add random groups
routes %v% "mygroup" <- sample(letters[1:4], network.size(routes), replace = TRUE)


In [ ]:
# load world map data
world <- ggplot(map_data("world"), aes(x = long, y = lat)) +
  geom_polygon(aes(group = group), color = "grey65",
               fill = "#f9f9f9", size = 0.2)

# overlay network data to map
x <- ggnetworkmap(
  world, routes, size = 4, great.circles = TRUE,
  node.group = mygroup, segment.color = "steelblue",
  ring.group = degree, weight = degree
)

plot(x)

We can see in the figure above the difference of level of the number of routes that pass in each airport or until finding points of greater centrality, that would affect the whole system. In the analysis of the synthetic data, we verified that the airport with the highest degree is Brasília (SBBR) and the route most used to reach the other points of the network is the route between São Paulo and Brasília (SBGR - SBBR).

In [ ]:
#Betweenness - Intermediation (Edge):
# It is a measure that quantifies the number of times an ARESTA acts as
#bridge along the shortest path between two other vertices.
graph <- network_to_igraph(routes)
b = edge.betweenness(graph)

#show de most busy route betweenn two airports
E(graph)[max(b)]

In [ ]:
#show de airport with max degree
network.vertex.names(routes)[which(routes %v% "degree" == max(degree(routes, gmode = "digraph")))]

# 6. Pattern Mining

In [ ]:
source("https://raw.githubusercontent.com/eogasawara/mylibrary/master/myGraphics.R")
loadlibrary("arules")
loadlibrary("arulesViz")
plot_size(4, 3)

The association rule is a machine learning method with the purpose of identifying rules of association between variables using some measures of interest.

One of the most used algorithms is the apriori algorithm, proposed by Agrawal et al (1993). This method assumes categorical data (does not work with numeric data), and was used initially in the analysis of the shopping basket of the supermarkets to determine how the items purchased by the customers were related.

In our example, we randomly assigned twenty-three aircraft models of different manufacturers. From this list, we generated a synthetic database representing three thousand flights made with these aircrafts, also indicating the age of the aircraft (how long in service) and the indication of delay of each flight. Our objective is to verify if there is association rule between the models of the aircraft and time in which it is in service, with the fact of generating delays.

In [ ]:
#read source data
aircraft <- read.csv("aircraft.csv")
summary(aircraft)

In [ ]:
# create random delays and its aircrafts information
set.seed(3000)
flights <- data.frame(
  aircraft = sample(aircraft[2:23, ]$aircraft, 3000, replace = TRUE),
  model = sample(aircraft[2:23, ]$model, 3000, replace = TRUE),
  age = sample(c("<5",">5"), 3000, replace = TRUE),
  delayed = sample(c("yes","no"), 3000, replace = TRUE)
)

#convert to factor
flights$aircraft <-  as.factor(flights$aircraft )
flights$model <-  as.factor(flights$model )
flights$age <-  as.factor(flights$age )
flights$delayed <-  as.factor(flights$delayed )
head(flights)

In [ ]:
# find association rules with default settings
rules <- apriori(flights, parameter= list(supp=0.4, conf=0.4))
inspect(rules)

In [ ]:
# rules with rhs containing "Delayed" only
rules <- apriori(flights,  parameter = list(minlen=2, supp=0.005, conf=0.7),
                 appearance = list(rhs=c("delayed=no", "delayed=yes"),default="lhs"),
                 control = list(verbose=F))

rules.sorted <- sort(rules, by="lift")
inspect(rules.sorted)

In [ ]:
# find redundant rules
subset.matrix <- is.subset(rules.sorted, rules.sorted)
subset.matrix[lower.tri(subset.matrix, diag=T)] <- NA
redundant <- colSums(subset.matrix, na.rm=T) >= 1
which(redundant)

# remove redundant rules
rules.pruned <- rules.sorted[!redundant]
inspect(rules.pruned)

In [ ]:
plot(rules)
plot(rules, method="graph", control=list(type="items"))
plot(rules, method="paracoord", control=list(reorder=TRUE))

After defining minimum levels of support and trust for the analysis, we found 5 association rules for cases where there was a delay, showing which aircraft models and time in service were related to these delays. The second figure above shows associated delay information, in the centre, the rules, represented by the spheres. the size of each sphere is proportional to the support and confidence of the information in relation to the delay.

# 7. Regression

In [ ]:
source("https://raw.githubusercontent.com/eogasawara/mylibrary/master/myGraphics.R")
loadlibrary("tidyverse")
loadlibrary("ggpubr")
loadlibrary("tidyverse")
loadlibrary("ggplot2")
plot_size(4, 3)

Regression analysis studies the relationship between a variable called the variable dependent variables and other variables called independent variables. The relation between them is represented by a mathematical model if association between the variables and can provide prediction of future behavior. One of the regression models, the simple linear regression model, defines a linear relationship between the dependent variable and a variable independent. If multiple independent variables are involved, it would be called a multiple linear regression model. This prediction can be achieved through a study involving the equation of (y, dependent or response) and independent (x, also known as prognosis) variables.

We set up a dataset with ten flight arrivals, with information representing time between the touch of tires on the ground and another field representing the time delay recorded on the flight. The purpose here is to know if there is a relationship between the two pieces of information.

In [ ]:
wheelson =    c(20,-10,  5,  2,  8, 30, -10, 5, 4, 6)

timedelayed = c(30,  5, 15, 10, 15, 20,  0, 25, 10, 18)

# single dataset combining multiple columns
df=data.frame(wheelson, timedelayed)
df

In [ ]:
theme_set(theme_pubr())

model <- lm(wheelson ~ timedelayed, data = df)
model

In [ ]:

ggplot(df, aes(timedelayed, wheelson)) +
  geom_point() +
  stat_smooth(method = lm)

From the graph we can visualize that there is a relationship between the variables, and that possibly a forecasting with a good level of accuracy, the total delay times of a flight through the delay between the tire touch time on the ground and the arrival at the gate. And, if true, as in the hypothetical case presented, it could indicate a problem in the operation of the airport, where the ground time since arrival includes the total time of delay, in cases of delay.